# Otimizando seu Vetor de Características

<div style="text-align:justify">
Vamos apresentar um conjunto de técnicas que ajudam a compreender a relevância das features utilizadas para treinar seu modelo de machine learning, em seguida; vamos mostrar como o iFood vem enfrentando desafios para fazer recomendação de restaurantes e como as técnicas discutidas nesta palestra tem ajudado a direcionar os próximos passos.
</div>

### Por que Otimizar?

- Saber onde buscar novas features
- Compreender a relação entre as features e os resultados obtidos

### Principais Técnicas

- Principal Component Analysis
- Recursive Feature Elimination
- Análise de Variância

### Pacotes Básicos

In [1]:
from sklearn.datasets import load_iris

import pandas as pd
import numpy as np

### Dataset Exemplo

In [2]:
iris = load_iris()
dataPD = pd.DataFrame(data=np.c_[iris['data'], iris['target']], \
                     columns= iris['feature_names'] + ['target'])

dataPD.head(5)

,sepal length (cm),sepal width (cm),petal length (cm),petal width (cm),target
0,5.1,3.5,1.4,0.2,0.0
1,4.9,3.0,1.4,0.2,0.0
2,4.7,3.2,1.3,0.2,0.0
3,4.6,3.1,1.5,0.2,0.0
4,5.0,3.6,1.4,0.2,0.0


### Principal Component Analysis PCA

<div style="text-align:justify">
PCA usa álgebra linear para transformar o conjunto de dados em uma forma compactada. Geralmente isso é chamado de técnica de redução de dados. Uma propriedade do PCA é que você pode escolher o número de dimensões ou o componentes principais no resultado transformado.
<div>

In [3]:
from sklearn.decomposition import PCA

array = dataPD.values
X = array[:,0:4]
Y = array[:,4]

# feature extraction
pca = PCA(n_components=2)
fit = pca.fit(X)

# summarize components
print("Explained Variance: %s") % fit.explained_variance_ratio_
print(fit.components_)

Explained Variance: [0.92461621 0.05301557]
[[ 0.36158968 -0.08226889  0.85657211  0.35884393]
 [ 0.65653988  0.72971237 -0.1757674  -0.07470647]]


### Recursive Feature Elimination (RFE)

<div style="text-align:justify">
RFE funciona removendo recursivamente os atributos e construindo um modelo nos atributos que permanecem. A precisão do modelo é usada para identificar quais atributos (e combinação de atributos) contribuem mais para prever o atributo objetivo.
</div>

In [7]:
from sklearn.feature_selection import RFE
from sklearn.svm import SVC

array = dataPD.values
X = array[:,0:4]
Y = array[:,4]

# feature extraction
model = SVC(kernel="linear")
rfe = RFE(model, 2)
fit = rfe.fit(X, Y)

# summarize results
print("Num Features: %d") % fit.n_features_
print("Selected Features: %s") % fit.support_

Num Features: 2
Selected Features: [False False  True  True]


<div style="text-align:justify">
RFE permite compreender um pouco mais a relevância das features, mas não oferecem muito entendimento sobre os resultados obtidos em algumas situações:
</div>

- Qual algoritmo devo usar quando dois algoritmos produzem resultados diferentes?
- Por que isso acontece?
- Onde devo buscar novas features?

Um ponto a ser considerado no RFE é o desempenho, quando o volume de dados for suficientemente grande.

### Análise de Variância

<div style="text-align:justify">
A análise de variância (ANOVA) é uma técnica estatística que pode ser utilizada para avaliar o erro de medição e outras fontes de variabilidade dos dados. Ela permite:
</div>

- Estimar a variância intra e entre features
- Evidenciar a relação entre as features, para uma análise mais detalhada

In [5]:
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2

X, y = iris.data, iris.target

selector = SelectKBest(chi2, k=2)
selector.fit(X, y)

X_new = selector.transform(X)
X_new.shape
print "Indices: %s" % (selector.get_support(indices=True))
print "Scores: %s" % (selector.scores_)

Indices: [2 3]
Scores: [ 10.81782088   3.59449902 116.16984746  67.24482759]


Pontos a serem observados:

- Features categóricas com one hot encoding
- O atributo de início exerce pouca influência sobre os resultados

# Estudo de Caso

<div style="text-align:justify">
O iFood está desenvolvendo e testando um recomendador de restaurantes. Vamos apresentar alguns dos desafios e dos resultados obtidos até agora com a seleção e análise de features.
</div>

### Metodologia

- Spark possui uma análise chi-square, mas... desenvolvemos um cálculo ANOVA
- Aplicamos uma análise de variância
- Analisamos a correlação entre as features
- Verificação gráfica das features mais e menos relevantes

### Resultados ANOVA

<img src="anova_ifood.png">

### Análise de Correlações

<div style="text-align:justify">
A análise de correlação permite verificar se duas variáveis estão correlacionadas, por exemplo: Gasto com publicidade X Volume de vendas. Na correlação existem dois fatores importantes a serem observados: o grau de correlação e a direção.
</div>

<img src="ancorr_ifood.png">

Dentre as correlações encontradas podemos mencionar:

- Correlação negativa entre as features 7 e 8 (plataforma de compra)
- Correlação positiva entre as features 0 e 1 (valor do pedido e taxa de entrega)

### Projeção de Features

<img src="analise_corr_ifood.png">

### Decisões

- Priorizar a análise de variância para melhorar o entendimento das features
- Deixar a redução de dimensionalidade (PCA) e validação do modelo (RME) para as próximas iterações

### Próximos Passos

- Avaliar os resultados do modelo que está sendo testado em produção
- Usar esses resultados como baseline atualizado para aplicar PCA e RME
- Selecionar novas features no contexto das features (12, 13, 2, 3 e 9)
- Repetir as análises para as novas features
- Incrementar modelo e iterar